In [1]:
import cv2 
import os
import matplotlib.image as mpimg
import albumentations as A
# imgaug uses matplotlib backend for displaying images
%matplotlib inline

import pandas as pd

Функция для получения фреймов(картинок) из видео и функция для создания дополнительных папок

In [2]:
def get_images(input_video):
    cap = cv2.VideoCapture(input_video)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    success,image = cap.read()
    count = 0
    while success:
        cv2.imwrite("images/%d.jpg" % count, image)     # save frame as JPEG file      
        success,image = cap.read()
        count += 1
    return height,width
    
def create_folder(name_path):
    if not os.path.exists(name_path):
        os.makedirs(name_path)

Глобальные переменные 

In [3]:
input_video='test_video_2.mkv'
default_images='images/'
aug_images='aug/'
resize_images='resize/'
different_images='difference/'
CSV='test_video_2.csv'
BOX_COLOR = (255, 0, 0) # Red

Создаём папки и получаем конфигурацию видео (высота и широта кадров)

In [4]:
create_folder(resize_images)
create_folder(default_images)
create_folder(aug_images)
create_folder(different_images)

height,width=get_images(input_video)
print(int(height),int(width))

1080 1920


Рассмотрим данные в csv файле

In [5]:
df = pd.read_csv(CSV)
print(df.head())

   frame        x0        y0        x1        y1
0      0  0.261465  0.128091  0.552427  0.303706
1      0  0.195457  0.252028  0.640548  0.700904
2      0  0.621903  0.251293  0.797448  0.489561
3      0  0.050217  0.258079  0.358569  0.479849
4      0  0.871230  0.250931  0.933671  0.327532


Функция для трансформации датафрейма. Относительные точки переводим в реальные.

In [6]:
def transform_df(df):
    df['frame'] = df['frame'] .apply(lambda x: str(x)+".jpg")
    df['x0'] = df['x0'] .apply(lambda x: round(x*width))
    df['x1'] = df['x1'] .apply(lambda x: round(x*width))
    df['y0'] = df['y0'] .apply(lambda x: round(x*height))
    df['y1'] = df['y1'] .apply(lambda x: round(x*height))
    return df

In [7]:
detections=transform_df(df)
detections.head()

,frame,x0,y0,x1,y1
0,0.jpg,502,138,1061,328
1,0.jpg,375,272,1230,757
2,0.jpg,1194,271,1531,529
3,0.jpg,96,279,688,518
4,0.jpg,1673,271,1793,354


Функции для визуализации bounding boxes на изображении

In [8]:
def visualize_bbox(img, bbox, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, x_max, y_max = bbox
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    return img

def visualize(image, bboxes):
    img = image.copy()
    for bbox in bboxes:
        img = visualize_bbox(img, bbox)
    return img

Функция для преобразования изображения в изображения с bounding box и сохранением их в отдельной папке. Вторая функция для создания видео с bounding box

In [9]:
def image_aug(df, images_path,output_path):
    # create data frame which we're going to populate with augmented image info
    grouped = df.groupby('frame')
    
    for filename in df['frame'].unique():
    #   get separate data frame grouped by file name
        group_df = grouped.get_group(filename)
        group_df = group_df.reset_index()
        group_df = group_df.drop(['index'], axis=1)   
    #   read the image
        image = mpimg.imread(images_path+filename)
    #   get bounding boxes coordinates and write into array        
        bb_array = group_df.drop(['frame'], axis=1).values
    #   pass the array of bounding boxes coordinates to the imgaug library
        img=visualize(image, bb_array)
        mpimg.imsave(output_path+filename,img)

def save_aug():
    os.system("ffmpeg -r 10 -i aug/%d.jpg -vcodec mpeg4 -y output_aug.mp4")

Рисуем bounding box на всех изображениях и сохраненяем видео. Получем фйал - output_aug.mp4

In [10]:
image_aug(detections,default_images,aug_images)
save_aug()

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.102)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolb

Функция для преобразования изображения с разрешением 416х416 и аннотацие данных, а так же сохранение видео с разрешением 416х416 название - output_aug_resized.mp4

In [11]:
height, width = 416, 416

def image_aug_resize(df, images_path,output_path):
    # create data frame which we're going to populate with augmented image info
    grouped = df.groupby('frame')
    transform = A.augmentations.crops.transforms.CenterCrop(height, width, p=1.0)
    for filename in df['frame'].unique():
    #   get separate data frame grouped by file name
        group_df = grouped.get_group(filename)
        group_df = group_df.reset_index()
        group_df = group_df.drop(['index'], axis=1)   
    #   read the image
        image = mpimg.imread(images_path+filename)
    #   get bounding boxes coordinates and write into array        
        bb_array = group_df.drop(['frame'], axis=1).values
    #   pass the array of bounding boxes coordinates to the imgaug library
        img=transform(image=visualize(image, bb_array))['image']
        mpimg.imsave(output_path+filename,img)

def save_resized():
    os.system("ffmpeg -r 10 -i resize/%d.jpg -vcodec mpeg4 -y output_aug_resized.mp4")

In [12]:
image_aug_resize(detections,default_images,resize_images)
save_resized()

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.102)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolb

Функция для взятия отличающиеся изображений(беру кадр каждые 2 секунды)

In [13]:
def get_nth_frame(videoFile,path):
    cap = cv2.VideoCapture(videoFile)
    success, image = cap.read()
    seconds = 2
    fps = cap.get(cv2.CAP_PROP_FPS) 
    multiplier = fps * seconds

    while success:
        frameId = int(round(cap.get(1))) 
        success, image = cap.read()
        if success:
            if frameId % multiplier == 0:
                cv2.imwrite(path+"/frame%d.jpg" % frameId, image)
    cap.release()
    cv2.destroyAllWindows()
    print("Complete")

Сохраняем отличающиеся изображения

In [14]:
video='output_aug_resized.mp4'
get_nth_frame(video,different_images)

Complete


In [15]:
import shutil
def remove_path(file_path):
    if os.path.exists(file_path):
        shutil.rmtree(file_path)
        print("folder deleted")
    else:
        print("The system cannot find the file specified")

def remove_video(video):
    os.remove(video)
    print("video deleted")

    

In [16]:
aug_video="output_aug.mp4"
res_video="output_aug_resized.mp4"

Если необходимо, то удаляем ранее созданные файлы и папки

In [17]:
remove_path(default_images)
remove_path(aug_images)
remove_path(resize_images)

folder deleted
folder deleted
folder deleted


In [18]:
remove_video(aug_video)
remove_video(res_video)

video deleted
video deleted


In [19]:
remove_path(different_images)

folder deleted
